# Assignment 1
You should submit the **UniversityNumber.ipynb** file and your final prediction file **UniversityNumber.test.txt** to moodle. Make sure your code does not use your local files and that the results are reproducible. Before submitting, please **1. clean all outputs and 2. run all cells in your notebook and keep all running logs** so that we can check.

# Installation of libraries/packages if needed

In [1]:
%pip install gensim==4.3.2
%pip install nltk==3.8.1
%pip install scikit-learn==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


# Imports

In [2]:
from datetime import datetime
from nltk.lm import Vocabulary, MLE, Laplace, Lidstone
from nltk.lm.api import LanguageModel
from nltk.lm.counter import NgramCounter
from nltk.util import bigrams, ngrams, trigrams
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import precision_score, recall_score, f1_score
from typing import List, Union

In [3]:
RANDOM_STATE = 3361
MAX_ITER = 1000
UID = '<YOUR UID HERE>'

# 1 $n$-gram Language Model

In [4]:
!mkdir -p data/lm
!wget -O data/lm/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/train.txt
!wget -O data/lm/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/dev.txt
!wget -O data/lm/test.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/test.txt

--2024-02-17 09:31:42--  https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8385238 (8.0M) [text/plain]
Saving to: ‘data/lm/train.txt’

data/lm/train.txt   100%[===================>]   8.00M  --.-KB/s    in 0.09s   

2024-02-17 09:31:42 (90.9 MB/s) - ‘data/lm/train.txt’ saved [8385238/8385238]

--2024-02-17 09:31:42--  https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/lm/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting r

## 1.1 Building vocabulary

### Code

In [5]:
# read the train.txt file and append to the list train_corpus
with open('data/lm/train.txt', 'r') as file:
    sentences_training = file.readlines()
    for i, text in enumerate(sentences_training):
        text_list = text.split()
        text_list.insert(0, '<START>')
        text_list.append('<END>')
        sentences_training[i] = text_list
words = [word for sublist in sentences_training for word in sublist]

In [6]:
# this vocab variable includes the words inside train.txt and <UNK>
vocab = Vocabulary(words, unk_cutoff=3)
print(datetime.now(), f'Vocabulary length = {len(vocab)}')

2024-02-17 09:31:43.941808 Vocabulary length = 26601


### Discussion

For the n-gram models, the number of parameters is of order $O(|V|^n)$, which grows rapidly under a polynomial relationship when the vocabulary size $|V|$ is large.

Specifically, for the case of this bigram, the vocabulary size $|V|$ is $26601$. Excluding the special tokens `<START>` and `<END>`, the vocabulary size is $26599$, which there are a total of $26599^2 = 707506801$ possible combinations of these words inside te vocabulary. Including the combinations of the `<START> *` and the `* <END>`, there are a total of $26599^2 + 26599 * 2 = 707559999$ parameters involved for each sentence, each representing a bigram from any two combinations of the vocabulary words and the `<START>` and `<END>` tokens.

From the example above, it is clear that the number of parameters inside the n-gram models suffer from the curse of dimensionality. As the dimension $n$ increases, the number of features inside a sentence grows rapidly, which makes the training data sparse. The computational power needed to perform subsequent tasks in training the n-gram model takes significantly more time as the vocabulary size increases, since the time complexity is $O(|V|^n)$ for operations involving all parameters and features of the dataset.

## 1.2 $n$-gram Language Modeling

### Code for Bigram

In [7]:
# read the dev.txt file
with open('data/lm/dev.txt', 'r') as file:
    sentences_dev = file.readlines()
    for i, text in enumerate(sentences_dev):
        text_list = text.split()
        text_list.insert(0, '<START>')
        text_list.append('<END>')
        sentences_dev[i] = text_list

In [8]:
# bigram model - training set
bigram_text_training = []
training_bigrams = []
for sentence in sentences_training:
    sentence = list(vocab.lookup(sentence))
    bigram_pairs = list(bigrams(sentence))
    bigram_text_training.append(bigram_pairs)
    for bigram in bigram_pairs:
        training_bigrams.append(bigram)
bigram_model = MLE(order=2, vocabulary=vocab)
print(datetime.now(), 'training bigram model...')
bigram_model.fit(bigram_text_training, vocab)
training_perplexity = bigram_model.perplexity(training_bigrams)
print(f'Bigram perplexity on training set = {training_perplexity}')
# bigram model - development set
dev_bigrams = []
for sentence in sentences_dev:
    sentence = list(vocab.lookup(sentence))
    bigram_pairs = list(bigrams(sentence))
    for bigram in bigram_pairs:
        dev_bigrams.append(bigram)
dev_perplexity = bigram_model.perplexity(dev_bigrams)
print(f'Bigram perplexity on dev set = {dev_perplexity}')

2024-02-17 09:31:47.927439 training bigram model...
Bigram perplexity on training set = 76.93455019656903
Bigram perplexity on dev set = inf


### Code for Unigram

In [9]:
# unigram model - training set
unigram_text_training = []
training_unigrams = []
for sentence in sentences_training:
    sentence = list(vocab.lookup(sentence))
    unigrams = list(ngrams(sentence, n=1))
    unigram_text_training.append(unigrams)
    for unigram in unigrams:
        training_unigrams.append(unigram)
unigram_model = MLE(order=1, vocabulary=vocab)
print(datetime.now(), 'training unigram model...')
unigram_model.fit(unigram_text_training, vocab)
training_perplexity = unigram_model.perplexity(training_unigrams)
print(f'Unigram perplexity on training set = {training_perplexity}')
# unigram model - development set
dev_unigrams = []
for sentence in sentences_dev:
    sentence = list(vocab.lookup(sentence))
    unigrams = list(ngrams(sentence, n=1))
    for unigram in unigrams:
        dev_unigrams.append(unigram)
dev_perplexity = unigram_model.perplexity(dev_unigrams)
print(f'Unigram perplexity on dev set = {dev_perplexity}')

2024-02-17 09:32:33.071718 training unigram model...
Unigram perplexity on training set = 888.8280671470116
Unigram perplexity on dev set = 815.9305409683114


### Discussion

For the bigram language model, the perplexity on the training set is $76.9$ while the perplexity on the dev set is `infinity`.

For the unigram language model, the perplexity on the training set is $888.8$ while the perplexity on the dev set is $815.9$.

One observation from the results is that the bigram perplexity on the dev set is infinity. The bigram model has not seen some instances in the dev set as they are not present in the training set. Thus, the output for the corresponding count/probability will be 0. Consequently, the term $\log_2 p(x^{(i)})$ will tend to negative infinity, causing the perplexity to become positive infinity.

In the unigram language model, the perplexity on the dev set is slightly lower than the perplexity on the training set. This implies that the effective vocabulary of the model is lower in the dev set compared to the training set. This maybe caused by pure luck due to the combination of words and sentences present in the dev set. This model is able to more accurately predict the next word under the dev set compared to the training set.

Comparing the perplexity on the training set for the bigram and unigram model, we can see that the bigram has a much lower perplexity than the unigram. This is because the assigned probabilities for a specific sentence in the bigram is higher, which causes the negative log-probability, thus the perplexity, to be lowered. Under the training set environment which is seen by the model, the bigram model is able to more accurately predict the next probable word than the unigram.

It is also notable that, based on the experimental results, the bigram model is overfitting compared to the unigram model. From the results above, the bigram model achieves a low training perplexity but has an infinitely large validation perplexity, while the magnitude of the unigram perplexity is similar in both the training and dev set. This is because the bigram model exhausts all the combinatorial possibilities inside the vocabulary, which introduces the sparsity problem in the dataset containing the presence of each bigram for each sentence. As the bigram model is overfitting to the training set, it cannot generalize well to the development set, as compared to the unigram model.

## 1.3 Smoothing

### 1.3.1 Add-one (Laplace) smoothing

### Code

In [10]:
# bigram model - training set
bigram_text_training = []
training_bigrams = []
for sentence in sentences_training:
    sentence = list(vocab.lookup(sentence))
    bigram_pairs = list(bigrams(sentence))
    bigram_text_training.append(bigram_pairs)
    for bigram in bigram_pairs:
        training_bigrams.append(bigram)
bigram_model = Laplace(order=2, vocabulary=vocab)
print(datetime.now(), 'training bigram model with add-one smoothing...')
bigram_model.fit(bigram_text_training, vocab)
training_perplexity = bigram_model.perplexity(training_bigrams)
print(f'Bigram perplexity on training set = {training_perplexity}')
# bigram model - development set
dev_bigrams = []
for sentence in sentences_dev:
    sentence = list(vocab.lookup(sentence))
    bigram_pairs = list(bigrams(sentence))
    for bigram in bigram_pairs:
        dev_bigrams.append(bigram)
dev_perplexity = bigram_model.perplexity(dev_bigrams)
print(f'Bigram perplexity on dev set = {dev_perplexity}')

2024-02-17 09:32:50.025905 training bigram model with add-one smoothing...
Bigram perplexity on training set = 1442.5940705370795
Bigram perplexity on dev set = 1660.2006059691935


#### Discussion

The perplexity of the bigram model with add-one smoothing on the training set and dev set are $1442.6$ and $1660.2$ respectively.

One difference in the perplexity with add-one smoothing compared to the previous part is that the bigram model is now less overfitting, as the perplexity on the training set and dev set is similar in magnitude.

However, the bigram perplexity on the dev set decreases significantly (while that of the training set increases significantly). As bigrams with originally assigned probabilities of zero in the dev set are increased to the inverse of the vocabulary size, this introduces an uncertainty to the model as the cross-entropy decreases, which implies the predictions made by the bigram model is less random. This effectively lowers the perplexity in the dev set.

#### 1.3.2 Add-k smoothing

##### Code

In [11]:
def add_k_smoothing(k: float):
    # bigram model - training set
    bigram_text_training = []
    training_bigrams = []
    for sentence in sentences_training:
        sentence = list(vocab.lookup(sentence))
        bigram_pairs = list(bigrams(sentence))
        bigram_text_training.append(bigram_pairs)
        for bigram in bigram_pairs:
            training_bigrams.append(bigram)
    bigram_model = Lidstone(order=2, vocabulary=vocab, gamma=k)
    print(datetime.now(), f'training bigram model with add-k smoothing, k={k}...')
    bigram_model.fit(bigram_text_training, vocab)
    training_perplexity = bigram_model.perplexity(training_bigrams)
    print(f'Bigram perplexity on training set = {training_perplexity}')
    # bigram model - development set
    dev_bigrams = []
    for sentence in sentences_dev:
        sentence = list(vocab.lookup(sentence))
        bigram_pairs = list(bigrams(sentence))
        for bigram in bigram_pairs:
            dev_bigrams.append(bigram)
    dev_perplexity = bigram_model.perplexity(dev_bigrams)
    print(f'Bigram perplexity on dev set = {dev_perplexity}')

In [12]:
Ks = [1e-2, 5e-3, 1e-3]
for k in Ks:
    add_k_smoothing(k)

2024-02-17 09:33:21.672733 training bigram model with add-k smoothing, k=0.01...
Bigram perplexity on training set = 157.7053131615645
Bigram perplexity on dev set = 437.5697758029117
2024-02-17 09:33:54.535309 training bigram model with add-k smoothing, k=0.005...
Bigram perplexity on training set = 129.34773785420956
Bigram perplexity on dev set = 416.66775227938956
2024-02-17 09:34:25.234471 training bigram model with add-k smoothing, k=0.001...
Bigram perplexity on training set = 94.95746093615871
Bigram perplexity on dev set = 432.42871956575476


##### Discussion

In this optimization experiment, the values $k \in \{0.01, 0.005, 0.001\}$ are chosen.

For $k = 0.01$, the training and validation perplexity are $157.7$ and $437.6$ respectively.

For $k = 0.005$, the training and validation perplexity are $129.3$ and $416.7$ respectively.

For $k = 0.001$, the training and validation perplexity are $95.0$ and $432.4$ respectively.

From these values of $k$, we can infer that the validation perplexity achieves a local minimum between $0.01$ and $0.001$, as the validation perplexity plotted against these values of $k$ exhibit a U-shaped curve. Thus, the optimized value of $k$ is chosen to be $0.005$.

Compared to the add-one smoothing, both the training and validation perplexity decreases in the add-k smoothing bigram model. The difference in results between this sub-question and the previous sub-question is due to the probabilities assigned to unseen bigrams. Under add-k smoothing, the probabilities are only slightly altered, whereas the probabilities may be significantly changed under add-one smoothing.

Another reason to explain why the add-k smoothing is better is because the value of $k$ can be alternatively viewed as the regularization strength of the objective function of the model. When $k$ increases, the strength of regularization increases which results in a simpler model, explaining the case where the training and validation perplexity is significantly larger in the case of add-one smoothing. As $k$ decreases, the regularization strength decreases which reduces the training perplexity. The validation perplexity, on the other hand, can generalize the best to the unseen bigrams when the strength of regularization is not as intense in the add-one smoothing case. This also justifies the use of a small value of $k$ in the add-k smoothing.

### 1.3.3 Linear Interpolation

#### Code

In [13]:
# define a new class for the linear interpolation smoothing model
class LinearInterpolation(LanguageModel):
     """Class for providing linear-interpolation smoothed scores.

    In addition to the iniitalization arguments from BaseNgramModel,
    it also requres lambda1, lambda2 and lambda3 to determine the weights for interpolation.
    """

     def __init__(self,
               lambda1: float, lambda2: float,
               lambda3: Union[float, None],
               *args, **kwargs):
          super().__init__(order=3, *args, **kwargs)
          self.order = 3
          self.lambda1 = lambda1
          self.lambda2 = lambda2
          if lambda3 is None or (lambda1 + lambda2 + lambda3) != 1:
               self.lambda3 = 1 - self.lambda1 - self.lambda2
          else:
               self.lambda3 = lambda3

     def unmasked_score(self, word, context=None):
          trigram_unmasked_score = self.context_counts(context).freq(word)
          bigram_unmasked_score = self.context_counts((context[-1],)).freq(word)
          unigram_unmasked_score = self.counts.unigrams.freq(word)
          return (
               self.lambda1 * trigram_unmasked_score +
               self.lambda2 * bigram_unmasked_score +
               self.lambda3 * unigram_unmasked_score
          )

In [14]:
# prepare unigrams for training use
unigram_text_training = []
training_unigrams = []
for sentence in sentences_training:
    sentence = list(vocab.lookup(sentence))
    unigrams = list(ngrams(sentence, n=1))
    unigram_text_training.append(unigrams)
    for unigram in unigrams:
        training_unigrams.append(unigram)

# prepare bigrams for training use
bigram_text_training = []
training_bigrams = []
for sentence in sentences_training:
    sentence = list(vocab.lookup(sentence))
    bigram_pairs = list(bigrams(sentence))
    bigram_text_training.append(bigram_pairs)
    for bigram in bigram_pairs:
        training_bigrams.append(bigram)

# prepare trigrams for training use
trigram_text_training = []
training_trigrams = []
for sentence in sentences_training:
    sentence.insert(0, '<START>')
    sentence = list(vocab.lookup(sentence))
    trigram_pairs = list(trigrams(sentence))
    trigram_text_training.append(trigram_pairs)
    for trigram in trigram_pairs:
        training_trigrams.append(trigram)

# prepare trigrams for dev use
dev_trigrams = []
for sentence in sentences_dev:
    sentence.insert(0, '<START>')
    sentence = list(vocab.lookup(sentence))
    trigram_pairs = list(trigrams(sentence))
    for trigram in trigram_pairs:
        dev_trigrams.append(trigram)

In [15]:
counter = NgramCounter(unigram_text_training + bigram_text_training + trigram_text_training)

In [16]:
def linear_interpolation_model(λ1: float, λ2: float, λ3: float):
    interpolation_model = LinearInterpolation(lambda1=λ1, lambda2=λ2, lambda3=λ3,
                                              vocabulary=vocab, counter=counter)
    print(datetime.now(), f'training linear interpolation model with λ1={λ1}, λ2={λ2}, λ3={λ3}...')
    interpolation_model.fit(trigram_text_training, vocab)
    training_perplexity = interpolation_model.perplexity(training_trigrams)
    print(f'perplexity on training set = {training_perplexity}')
    dev_perplexity = interpolation_model.perplexity(dev_trigrams)
    print(f'perplexity on dev set = {dev_perplexity}')

In [17]:
# training on the linear interpolation model
linear_interpolation_model(0.2, 0.3, 0.5)

2024-02-17 09:35:19.831368 training linear interpolation model with λ1=0.2, λ2=0.3, λ3=0.5...
perplexity on training set = 25.107091102903894
perplexity on dev set = 283.037409372196


In [18]:
# optimization
lambdas_list = [
    (0.2, 0.2, 0.6),
    (0.2, 0.4, 0.4),
    (0.2, 0.5, 0.3),
    (0.2, 0.6, 0.2),
    (0.1, 0.6, 0.3),
    (0.1, 0.5, 0.4),
    (0.1, 0.4, 0.5),
    (0.1, 0.3, 0.6)
]
for lambdas in lambdas_list:
    λ1, λ2, λ3 = lambdas
    linear_interpolation_model(λ1, λ2, λ3)

2024-02-17 09:36:01.201851 training linear interpolation model with λ1=0.2, λ2=0.2, λ3=0.6...
perplexity on training set = 26.861129817310378
perplexity on dev set = 307.2068445979232
2024-02-17 09:36:40.062359 training linear interpolation model with λ1=0.2, λ2=0.4, λ3=0.4...
perplexity on training set = 23.664553497096755
perplexity on dev set = 269.4067227510323
2024-02-17 09:37:19.007396 training linear interpolation model with λ1=0.2, λ2=0.5, λ3=0.3...
perplexity on training set = 22.447063943636785
perplexity on dev set = 264.050881611073
2024-02-17 09:37:58.713523 training linear interpolation model with λ1=0.2, λ2=0.6, λ3=0.2...
perplexity on training set = 21.400428089985592
perplexity on dev set = 268.32456638867234
2024-02-17 09:38:38.479375 training linear interpolation model with λ1=0.1, λ2=0.6, λ3=0.3...
perplexity on training set = 31.19881408077877
perplexity on dev set = 262.8349689514711
2024-02-17 09:39:17.658898 training linear interpolation model with λ1=0.1, λ2=0.

In [19]:
# prepare the test sentences and trigrams
# read the test.txt file
with open('data/lm/test.txt', 'r') as file:
    sentences_test = file.readlines()
    for i, text in enumerate(sentences_test):
        text_list = text.split()
        text_list.insert(0, '<START>')
        text_list.append('<END>')
        sentences_test[i] = text_list

# prepare trigrams for test use
test_trigrams = []
for sentence in sentences_test:
    sentence.insert(0, '<START>')
    sentence = list(vocab.lookup(sentence))
    trigram_pairs = list(trigrams(sentence))
    for trigram in trigram_pairs:
        test_trigrams.append(trigram)

In [20]:
# fit the best optimized model on the test set
λ1, λ2, λ3 = 0.1, 0.6, 0.3
interpolation_model = LinearInterpolation(lambda1=λ1, lambda2=λ2, lambda3=λ3,
                                          vocabulary=vocab, counter=counter)
print(datetime.now(), f'training linear interpolation model with λ1={λ1}, λ2={λ2}, λ3={λ3}...')
interpolation_model.fit(trigram_text_training, vocab)
test_perplexity = interpolation_model.perplexity(test_trigrams)
print(f'perplexity on test set = {test_perplexity}')

2024-02-17 09:41:15.810189 training linear interpolation model with λ1=0.1, λ2=0.6, λ3=0.3...
perplexity on test set = 262.35175842315573


#### Discussion

For $\lambda_1 = 0.2$, $\lambda_2 = 0.3$, $\lambda_3= 0.5$, the perplexity on the training set and dev set are $25.1$ and $283.0$ respectively.

By optimizing the dev set on the hyperparameter sets $(\lambda_1, \lambda_2, \lambda_3) \in \{(0.2, 0.2, 0.6), (0.2, 0.4, 0,4), (0.2, 0.5, 0.3), (0.2, 0.6, 0.2), $
$(0.1, 0.6, 0.3), (0.1, 0.5, 0.4), (0.1, 0.4, 0.5), (0.1, 0.3, 0.6) \}$, the best hyperparameter set which minimizes the perplexity in the dev set is $(\lambda_1, \lambda_2, \lambda_3) = (0.1, 0.6, 0.3)$, with a training perplexity of $31.2$ and a perplexity of $262.8$ on the dev set.

The perplexity on the test set with the best hyperparameter set $(\lambda_1, \lambda_2, \lambda_3) = (0.1, 0.6, 0.3)$ is $262.4$.

The linear interpolation smoothing model performs much better than the add-k smoothing language model, as suggested by the decrease in both the perplexity on the training and dev set. In the previous sub-questions, the add-k smoothing is only performed on the bigram model, whereas the linear interpolation smoothing is performed between unigram, bigram and trigram models, which mitigates some deficiencies in the previous model.

The linear interpolation smoothing model can better capture the semantics inside each sentence. As some English phrases have three or more words, incorporating trigrams can capture these phrases, so that the model can recognize them through the trigram score given the previous context of the two words.

The optimal weights $(\lambda_1, \lambda_2, \lambda_3) = (0.1, 0.6, 0.3)$ suggest that a larger weight should be put on the bigram model, followed by the unigram and trigram model, meaning that bigram features contributes to a higher score, thus a lower perplexity of the model. The small weight on the trigram score captures the score contributed by the previous two words. The weight on the unigram score act as a backup to avoid the perplexity to become infinity, in case a word combination is not found in the training bigrams or trigrams.

From the experimental results above, we can see that the optimized linear interpolation smoothing model is not overfitting, as the perplexity on the test set is of similar magnitude compared to the one on the dev set.

##### 1.3.4 Optimization

#### Discussion

Bayesian Optimization is an example of a learning algorithm to find the optimal set of hyperparameters.

Bayesian Optimization is an optimization technique which aims to maximize or minimize the objective function (for instance, language model perplexity in this question). This technique takes a probabilistic approach to optimize a set of hyperparameters, keeping a memory of past hyperparameter combinations which can maximize the score/value of the objective function. The optimization technique starts with a prior probability distribution where we assume that all sets of hyperparameters perform equally well. After sufficient tuning, the posterior distribution will be updated based on different sets of hyperparameters, thus estimating the true form of the objective function well. Compared to traditional learning methods like grid search and randomized search, Bayesian Optimization has the advantage of not assuming the functional form of the objective function. By using a probabilistic approach, the effort in tuning the set of hyperparameters is lowered, as less unnecessary calls to the objective function is made.

# 2 Word Vectors

In [21]:
def load_embedding_model():
    """ Load GloVe Vectors
        Return:
            wv_from_bin: All 400000 embeddings, each length 50
    """
    import gensim.downloader as api
    wv_from_bin = api.load("glove-wiki-gigaword-200")
    print("Loaded vocab size %i" % len(list(wv_from_bin.index_to_key)))
    return wv_from_bin

wv_from_bin = load_embedding_model()

Loaded vocab size 400000


## 2.1 Find most similar word
Use cosine similarity to find the most similar word to each of these words. Report the most similar word and its cosine similarity.

* dog
* whale
* before
* however
* fabricate

In [22]:
words = ['dog', 'whale', 'before', 'however', 'fabricate']
for word in words:
    most_similar_word, similarity = wv_from_bin.most_similar(positive=word, topn=1)[0]
    print(f'The most similar to "{word}" is "{most_similar_word}" with cosine similarity {similarity}')

The most similar to "dog" is "dogs" with cosine similarity 0.8136862516403198
The most similar to "whale" is "whales" with cosine similarity 0.7918056845664978
The most similar to "before" is "after" with cosine similarity 0.8931248188018799
The most similar to "however" is "although" with cosine similarity 0.9336162805557251
The most similar to "fabricate" is "fabricating" with cosine similarity 0.618401825428009


## 2.2 Finding Analogies
Use vector addition and subtraction to compute target vectors for the analogies below. After computing each target vector, find the top three candidates by cosine similarity. Report the candidates and their similarities to the target vector.

- dog : puppy :: cat : ?:
- speak : speaker :: sing : ?:
- France : French :: England : ?:
- France : wine :: England : ?

In [23]:
analogies = [
    'dog:puppy::cat:?',
    'speak:speaker::sing:?',
    'France:French::England:?',
    'France:wine::England:?'
]
for analogy in analogies:
    word_list = re.split(r':+', analogy)
    w1, w2, w3 = word_list[0].lower(), word_list[1].lower(), word_list[2].lower()
    results = wv_from_bin.most_similar(positive=[w3, w2], negative=[w1], topn=3)
    print(f'For the analogy {analogy}, the top 3 candidate words with their similarities are: ')
    for i, result in enumerate(results):
        candidate_word, similarity = result
        print(f'{i+1}. {candidate_word}, similarity = {similarity}')
    print('----')

For the analogy dog:puppy::cat:?, the top 3 candidate words with their similarities are: 
1. puppies, similarity = 0.6142244935035706
2. kitten, similarity = 0.5919069647789001
3. kittens, similarity = 0.5758378505706787
----
For the analogy speak:speaker::sing:?, the top 3 candidate words with their similarities are: 
1. sang, similarity = 0.48609817028045654
2. chorus, similarity = 0.4438377618789673
3. singing, similarity = 0.4332594871520996
----
For the analogy France:French::England:?, the top 3 candidate words with their similarities are: 
1. english, similarity = 0.7599895596504211
2. british, similarity = 0.5879749059677124
3. scottish, similarity = 0.5616408586502075
----
For the analogy France:wine::England:?, the top 3 candidate words with their similarities are: 
1. tea, similarity = 0.540773868560791
2. wines, similarity = 0.5329136848449707
3. yorkshire, similarity = 0.49405309557914734
----


# 3 Sentiment analysis

In [24]:
!mkdir -p data/classification
!wget -O data/classification/train.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/train.txt
!wget -O data/classification/dev.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/dev.txt
!wget -O data/classification/test-blind.txt https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/test-blind.txt

--2024-02-17 09:42:46--  https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 738844 (722K) [text/plain]
Saving to: ‘data/classification/train.txt’

data/classification 100%[===================>] 721.53K  --.-KB/s    in 0.04s   

2024-02-17 09:42:46 (15.7 MB/s) - ‘data/classification/train.txt’ saved [738844/738844]

--2024-02-17 09:42:46--  https://raw.githubusercontent.com/ranpox/comp3361-spring2024/main/assignments/A1/data/classification/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:44

## 3.1 Logistic Regression

In [25]:
PATTERN = re.compile(r'([0-1])\t(.*)')

# read the dataset and preprocess it before model fitting
train_sentences = []
train_labels = []
# read the train.txt file
with open('data/classification/train.txt', 'r') as file:
    lines = file.readlines()
    for i, text in enumerate(lines):
        match = re.search(PATTERN, text)
        label, sentence = int(match.group(1)), match.group(2)
        train_sentences.append(sentence)
        train_labels.append(label)

In [26]:
dev_sentences = []
dev_labels = []
# read the dev.txt file
with open('data/classification/dev.txt', 'r') as file:
    lines = file.readlines()
    for i, text in enumerate(lines):
        match = re.search(PATTERN, text)
        label, sentence = int(match.group(1)), match.group(2)
        dev_sentences.append(sentence)
        dev_labels.append(label)

### Unigram Features

In [27]:
unigram_vec = CountVectorizer()
X_train = unigram_vec.fit_transform(train_sentences)
y_train = np.array(train_labels)

model = LogisticRegression(class_weight='balanced', max_iter=MAX_ITER, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

X_dev = unigram_vec.transform(dev_sentences)
y_pred = model.predict(X_dev)
y_dev = np.array(dev_labels)

precision = precision_score(y_dev, y_pred)
recall = recall_score(y_dev, y_pred)
f1 = f1_score(y_dev, y_pred)

print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'F1-score = {f1}')

Precision = 0.7866666666666666
Recall = 0.7972972972972973
F1-score = 0.7919463087248321


### Bigram Features

In [28]:
bigram_vec = CountVectorizer(ngram_range=(2, 2))
X_train = bigram_vec.fit_transform(train_sentences)
y_train = np.array(train_labels)

model = LogisticRegression(class_weight='balanced', max_iter=MAX_ITER, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

X_dev = bigram_vec.transform(dev_sentences)
y_pred = model.predict(X_dev)
y_dev = np.array(dev_labels)

precision = precision_score(y_dev, y_pred)
recall = recall_score(y_dev, y_pred)
f1 = f1_score(y_dev, y_pred)

print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'F1-score = {f1}')

Precision = 0.7104166666666667
Recall = 0.7680180180180181
F1-score = 0.7380952380952381


### GloVe Features

In [29]:
# assume GloVe features = average of the word vectors in a sentence
def get_glove_features(sentence: str) -> np.array:
    mean_vector = wv_from_bin.get_mean_vector([word.lower() for word in sentence.split()])
    return mean_vector

In [30]:
X_train = np.array([get_glove_features(sent) for sent in train_sentences])
y_train = np.array(train_labels)

model = LogisticRegression(class_weight='balanced', max_iter=MAX_ITER, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

X_dev = np.array([get_glove_features(sent) for sent in dev_sentences])
y_pred = model.predict(X_dev)
y_dev = np.array(dev_labels)

precision = precision_score(y_dev, y_pred)
recall = recall_score(y_dev, y_pred)
f1 = f1_score(y_dev, y_pred)

print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'F1-score = {f1}')

Precision = 0.7709251101321586
Recall = 0.7882882882882883
F1-score = 0.7795100222717148


Compare the performance of three types of features on dev set. Report the weighted average precision, recall and F1-score for each feature set.

| Feature | precision | recall | F1-score |
| ----------- | --------- | ------ | -------- |
| unigram     | 0.7867          | 0.7973       | 0.7919         |
| bigram      | 0.7104          | 0.7680       | 0.7381         |
| GloVe       | 0.7709          | 0.7883       | 0.7795         |

Overall, the performance for unigram features is the highest, followed by GloVe features, then bigram features.

All three features have the recall score greater than the precision score, which suggests that the logistic regression model has identified fewer false negatives than false positives.

One reason to account for the poor performance for the model with bigram features (as compared to other features) is that the feature matrix of the bigram features is very sparse. Based on the training corpus of sentences, there are a total of more than $67000$ bigram features. As there are only a few words in each sentence inside the training example, we would expect only 20-30 of these features having a value of `1`, with the other features having a value `0`. Thus, we may not have enough samples to train on specific bigram features, considering that there are tens of thousands of bigrams.

On the other hand, the model with unigram features slightly exceeds the performance of GloVe features, which suggests that the count and occurrence of each word is slightly better in correctly identifying positive samples, compared to the global representation/semantic meaning of each sentence as used in GloVe features.

## 3.2 Better Feature

In this sub-question, the following feature modifications are performed on the unigram/bigram model:
* Create a new feature which captures the count of negation words in a sentence (e.g. not, never and none).
* Create a new feature which captures the count of conjunctions in a sentence (e.g. but, yet, however).
* Create a new feature which captures the count of positive adjectives in a sentence, randomly sampled from the training dataset.
* Create a new feature which captures the count of negative adjectives in a sentence, randomly sampled from the training dataset.
* Include trigrams into consideration.
* Use a subset of features taking from unigrams, bigrams and trigrams.
* Use tf-idf weighting instead of the regular weighting using Maximum Likelihood Estimation (MLE).
* Disregard commonly appearing words which appear more than $n$ times in the training dataset, where $n \in \{200, 500, 800, 1000, 1200, 1500, 2000, 3000\}$.
* Disregard rare words which appear less than $n$ times in the training dataset, where $n \in \{2, 3, 4, 5, 10, 20, 50\}$.
* Disregard English stopwords.

In [31]:
# a new feature is created which captures the common negation words in English
NEGATION_WORDS_PATTERN = re.compile(r'\b(n(\'|o)t|never|nor|none|nothing|nowhere)\b', flags=re.IGNORECASE)

def get_negation_words_count(sentence: str) -> int:
    matches = re.findall(NEGATION_WORDS_PATTERN, sentence)
    return len(matches)

In [32]:
# a new feature is created which captures the common conjunctions with convey opposite meaning in English
CONJUNCTIONS_PATTERN = re.compile(r'\b(but|yet|however|(al)?though|(never|none)theless)\b', flags=re.IGNORECASE)

def get_conjunction_words_count(sentence: str) -> int:
    matches = re.findall(CONJUNCTIONS_PATTERN, sentence)
    return len(matches)

In [33]:
# a new feature is created to capture the occurrence of positive adjectives
POSITIVE_ADJ_PATTERN = re.compile(r'\b(charming|fascinating|astonishing|good|better|best|excellent|terrific|fantastic|vivid(ly)?|nice|expressive|funn(y|ier|iest)|beautiful|appealing|sophisticated|sweet|winning|fascinating|happy|glorious|inspiring|impressive|intriguing)\b', flags=re.IGNORECASE)

def get_positive_adj_count(sentence: str) -> int:
    matches = re.findall(POSITIVE_ADJ_PATTERN, sentence)
    return len(matches)

In [34]:
# a new feature is created to capture the occurrence of negative adjectives
NEGATIVE_ADJ_PATTERN = re.compile(r'\b(bad|worse|worst|horrible|silly|unoriginal|loose|imitation(s)?|clich(e|é)s|mediocre|bleak|incoheren(t|ce)|lack|odd(s)?|empty|evil|dull|predictable|bland|mess(y)?|disappoint(ed|ment)|dishonest|unappealing|awful|choppy|pointless|idiotic|ill(\-[a-z]+)?)\b', flags=re.IGNORECASE)

def get_negative_adj_count(sentence: str) -> int:
    matches = re.findall(NEGATIVE_ADJ_PATTERN, sentence)
    return len(matches)

After a series of optimization and fitting the transformed data to the training dataset (and the dev dataset for evaluation), the following transformations are applied to the original dataset:
* Create a new feature which captures the count of conjunctions in a sentence (e.g. but, yet, however).
* Create a new feature which captures the count of positive adjectives in a sentence, randomly sampled from the training dataset.
* Create a new feature which captures the count of negative adjectives in a sentence, randomly sampled from the training dataset.
* Use both unigrams and bigrams as features.
* Use tf-idf weighting.
* Disregard commonly appearing words which appear more than $1000$ times in the training dataset.

In [35]:
def apply_transformation_to_sentences(sentences: List,
                                      vec: Union[CountVectorizer, TfidfVectorizer],
                                      train: bool = True) -> np.array:
    if train:
        transformed_feats = vec.fit_transform(sentences)
    else:
        transformed_feats = vec.transform(sentences)
    # negation_words_feature = np.array([get_negation_words_count(sentence) for sentence in sentences])
    conjunction_words_feature = np.array([get_conjunction_words_count(sentence) for sentence in sentences])
    positive_adj_feature = np.array([get_positive_adj_count(sentence) for sentence in sentences])
    negative_adj_feature = np.array([get_negative_adj_count(sentence) for sentence in sentences])
    result = np.hstack((
                        transformed_feats.toarray(),
                        # negation_words_feature[:, np.newaxis],
                        conjunction_words_feature[:, np.newaxis],
                        positive_adj_feature[:, np.newaxis],
                        negative_adj_feature[:, np.newaxis],
                        ))
    return result

In [36]:
# use unigram
vec = TfidfVectorizer(ngram_range=(1, 2), max_df=1000)
X_train = apply_transformation_to_sentences(train_sentences, vec)
y_train = np.array(train_labels)

model = LogisticRegression(class_weight='balanced', max_iter=MAX_ITER, random_state=RANDOM_STATE)
model.fit(X_train, y_train)

X_dev = apply_transformation_to_sentences(dev_sentences, vec, train=False)
y_pred = model.predict(X_dev)
y_dev = np.array(dev_labels)

precision = precision_score(y_dev, y_pred)
recall = recall_score(y_dev, y_pred)
f1 = f1_score(y_dev, y_pred)

print(f'Precision = {precision}')
print(f'Recall = {recall}')
print(f'F1-score = {f1}')

Precision = 0.8070953436807096
Recall = 0.8198198198198198
F1-score = 0.8134078212290502


Based on the transformations applied, the performance on the dev set which the feature modifications has slightly improved. The precision, recall and f1-score obtained are $0.8071$, $0.8198$ and $0.8134$ respectively.

Finally, this model is applied to the test set and an output text file is generated.

In [37]:
# finally, read the test sentences and generate the predictions
test_sentences = []
test_labels = []
# read the dev.txt file
with open('data/classification/test-blind.txt', 'r') as file:
    test_sentences = file.readlines()
X_test = apply_transformation_to_sentences(test_sentences, vec, train=False)
y_pred = model.predict(X_test)
with open(f'{UID}.test.txt', 'w') as file:
    for i, sentence in enumerate(test_sentences):
        label = y_pred[i]
        file.write(f'{label}\t{sentence}')
    file.close()